# Building a Spam Filter with Naive Bayes

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80%. We expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

### Summary of Results
We created a highly accurate spam filter that achieved an accuracy of 98.29%, almost 20% higher than our goal. 

# Exploring the Dataset
We'll start by reading in the dataset and do a quick exploration of it.

In [56]:
# Read in the data
import pandas as pd
sms = pd.read_csv("SMSSpamCollection", sep="\t", header=None, names=["Label", "SMS"])

# Quick exploration of the data
print(sms.shape)
display(sms.head())
display(sms.tail())

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Label,SMS
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [2]:
sms.isnull().sum() # No missing values

Label    0
SMS      0
dtype: int64

In [3]:
# Percentage of spam and non-spam messages in our dataset
sms["Label"].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Above, we see that about 87% of the messages are ham and the remaining 13% are spam. This sample looks representative, since in practice most messages people receive are ham.

# Training and Test Set
We'll now split our dataset into a training set and a test set, where the training set accounts for 80% of the data and the test set for the remaining 20%.

In [4]:
# Randomize the dataset
sms_randomized = sms.sample(frac=1, random_state=1) # frac: fraction of axis items to return

# Calculate index for splitting
index = round(len(sms_randomized) * 0.8)

# Split into training and test sets
training_set = sms_randomized[:index].reset_index(drop=True)
test_set = sms_randomized[index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


Let's now analyze the percentage of spam and ham messages in each training and test set.

In [5]:
# Training set percentage of spam and non-spam messages
training_set["Label"].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [6]:
# Test set percentage of spam and non-spam messages
test_set["Label"].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Our training and test sets seem to have percentages close to what we have in the full dataset--87% ham and 13% spam. Let's now move on to cleaning the dataset.

# Data Cleaning
We'll need to perform a bit of data cleaning first before we can calculate all the probabilities required by the algorithm. We'll need to transform the dataset in a format that will allow us to extract the information we need.

We'll essentially be transforming the `SMS` column into multiple new columns, where each column represents a unique word from the vocabulary. More specifically, each column will show the frequency of that unique word for any given message.

## Letter Case and Punctuation
We'll begin with removing punctuations and bringing every letter to lowercase.

In [7]:
training_set["SMS"] = (training_set["SMS"].str.replace("\W", " ", regex=True) # \W :
                                        .str.lower())                    # NON digit/uppercase/lowercase/underscore
training_set["SMS"].head()

0                         yep  by the pretty sculpture
1        yes  princess  are you going to make me moan 
2                           welp apparently he retired
3                                              havent 
4    i forgot 2 ask ü all smth   there s a card on ...
Name: SMS, dtype: object

## Creating the Vocabulary
We'll now create a list containing all the unique words from our training set, where each word is represented as a string. 

In [8]:
training_set["SMS"] = training_set["SMS"].str.split()

# Append all words in the training set messages to vocabulary (contains duplicates)
vocabulary = []
for message in training_set["SMS"]:
    for word in message:
        vocabulary.append(word)

# Remove duplicate words in vocabulary by converting our List into Set, then convert it back to List
vocabulary = list(sorted(set(vocabulary)))

In [9]:
len(vocabulary)

7783

There are 7,783 unique words in all the messages from our training set.

# The Final Training Set
We'll now use `vocabulary` to transform the training dataset. We'll initialize a Dictionary where each key, value pair corresponds to a unique word from `vocabulary` and a List[ints] of length `training_set`. The indices in the List corresponds to the row indices in `training_set`, so each element in the List represents the number of times the unique word showed up in the SMS in `training_set`.

In [10]:
# Dictionary where each key is a unique word from the vocabulary
# and each value is a list of length training_set.
# Indices in the list correspond to the row indices in training_set.
word_counts_per_sms = {unique_word: [0] * len(training_set["SMS"]) for unique_word in vocabulary}

for index, sms in enumerate(training_set["SMS"]):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
# Convert Dictionary to DataFrame where the keys are on the column
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [12]:
# Concatenate the training set and the word counts DataFrame, both share the same row axis
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


# Calculating Constants First
Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.

The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages.

$$ P(Spam|w_1,w_2,...,w_n) ∝ P(Spam) \cdot \prod_{i=1}^{n} P(w_i|Spam) $$
$$ P(Ham|w_1,w_2,...,w_n) ∝ P(Ham) \cdot \prod_{i=1}^{n} P(w_i|Ham) $$
where
$$ P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{vocabulary}} $$
$$ P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{vocabulary}} $$

Note that: 
* $N_{Spam}$ is the total number of words in all the spam messages.
* $N_{Ham}$ is the total number of words in all the non-spam messages.
* $N_{vocabulary}$ is the total number of words in the vocabulary.
* $N_{w_i|Spam}$ is the number of times the word $w_i$ occurs in all the spam messages.
* $N_{w_i|Ham}$ is the number of times the word $w_i$ occurs in all the ham messages.

We'll also use Laplace smoothing ($\alpha = 1$).

We'll now calculate the constants: $P(Spam), \ P(Ham), \ N_{Spam}, \ N_{Ham}, \ N_{vocabulary}$.

In [13]:
# Filtering the spam and non-spam messages in the training set
spam_msgs = training_set_clean[training_set_clean["Label"] == "spam"]
ham_msgs = training_set_clean[training_set_clean["Label"] == "ham"]

# Probability that a message is spam/non-spam in the training set
p_spam = len(spam_msgs) / len(training_set_clean)
p_ham = len(ham_msgs) / len(training_set_clean)

# Total number of words in all the spam/non-spam messages
n_spam = (spam_msgs["SMS"].apply(len)).sum()
n_ham = (ham_msgs["SMS"].apply(len)).sum()

n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

# Calculating Parameters
Now that we have the constant terms calculated above, we'll now calculate for the parameters
$$ P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{vocabulary}} $$
$$ P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{vocabulary}} $$
for each word $w_i$ in the vocabulary.

In [14]:
# Store parameters in dictionaries
# where each key is a unique word in `vocabulary` and each value is initialized with 0
spam_params = dict.fromkeys(vocabulary, 0)
ham_params = dict.fromkeys(vocabulary, 0)

for word in vocabulary:
    # Number of times the word occurs in all spam messages
    n_w_given_spam = spam_msgs[word].sum()
    # Probability the word shows up given that a message is spam
    p_w_given_spam = (n_w_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    # Store the probability with the corresponding word in the spam dictionary 
    spam_params[word] = p_w_given_spam
    
    #Same idea as above for non-spam
    n_w_given_ham = ham_msgs[word].sum()
    p_w_given_ham = (n_w_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    ham_params[word] = p_w_given_ham

# Classifying a New Message
Now that we have all the parameters calculated, we can now create the spam filter. 

Recall that the spam filter utilizes these probabilities:
$$ P(Spam|w_1,w_2,...,w_n) ∝ P(Spam) \cdot \prod_{i=1}^{n} P(w_i|Spam) $$
$$ P(Ham|w_1,w_2,...,w_n) ∝ P(Ham) \cdot \prod_{i=1}^{n} P(w_i|Ham) $$
and we've finished calculating for $$ P(Spam), P(Ham), P(w_i|Spam), P(w_i|Ham) $$
stored in `p_spam`, `p_ham`, `spam_params`, `ham_params` respectively.


The spam filter is a function that:
* Takes in as input a new message ($w_1, w_2, ..., w_n$).
* Calculates $P(Spam|w_1, w_2, ..., w_n) \ and \ P(Ham|w_1, w_2, ..., w_n$).
* Compares the values of $P(Spam|w_1, w_2, ..., w_n) \ and \  P(Ham|w_1, w_2, ..., w_n$), and:
    * If $P(Ham|w_1, w_2, ..., w_n) > P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as ham.
    * If $P(Ham|w_1, w_2, ..., w_n) < P(Spam|w_1, w_2, ..., w_n$), then the message is classified as spam.
    * If $P(Ham|w_1, w_2, ..., w_n) = P(Spam|w_1, w_2, ..., w_n$), then the algorithm may request human help.

In [15]:
def classify(message):
    """
    Prints the probabilities that the message is spam/non-spam
    and the classification (spam/non-spam/need human help).
    
    Parameters:
        message(str):The string to classify.
    
    """
    
    # Clean message, transform to List[strs]
    message = (message.replace("\W", " ")
                      .lower()
                      .split())
    
    # Initialize probabilities with constants
    p_spam_given_msg = p_spam 
    p_ham_given_msg = p_ham
    
    # Look up each word in the message in the parameter dictionaries 
    # Looks up the calculation of the probability the word shows up given that a message is spam/non-spam
    #                         and the probability the word shows up given that a message is non-spam
    for word in message:
        if word in spam_params:
            p_spam_given_msg *= spam_params[word]
        if word in ham_params:
            p_ham_given_msg *= ham_params[word]
    
    # Prints probabilities and classification
    print("P(Spam|message):", p_spam_given_msg)
    print("P(Ham|message):", p_ham_given_msg)
    if p_ham_given_msg > p_spam_given_msg:
        print("Label: Ham")
    elif p_ham_given_msg < p_spam_given_msg:
        print("Label: Spam")
    else:
        print("Equal probabilities. Have a human classify this!")

In [16]:
classify("YOU WON!!! Claim your prize @ www.jfzhul.co")

P(Spam|message): 1.8375885649658594e-10
P(Ham|message): 2.607764643641013e-14
Label: Spam


In [17]:
classify("Secret party tonight! I'll text u the address l8r")

P(Spam|message): 2.462438934624288e-24
P(Ham|message): 2.968179728663378e-23
Label: Ham


# Measuring the Spam Filter's Accuracy
The results above look promising. Let's now apply our spam filter to the test set, which has 1,114 messages. 

Let's modify our `classify()` function to return labels instead of printing them, apply the function to the test set, and calculate the accuracy of the spam filter.

In [18]:
def classify_test_set(message):
    """
    Returns the classification label(spam/non-spam/need human help) of the message.
    
    Parameters:
        message(str):The string to classify.
    
    """
    
    # Clean message, transform to List[strs]
    message = (message.replace("\W", " ")
                      .lower()
                      .split())
    
    # Initialize probabilities with constants
    p_spam_given_msg = p_spam 
    p_ham_given_msg = p_ham
    
    # Look up each word in the message in the parameter dictionaries 
    # Looks up the calculation of the probability the word shows up given that a message is spam
    #                         and the probability the word shows up given that a message is non-spam
    for word in message:
        if word in spam_params:
            p_spam_given_msg *= spam_params[word]
        if word in ham_params:
            p_ham_given_msg *= ham_params[word]
    
    # Return classification
    if p_ham_given_msg > p_spam_given_msg:
        return "ham"
    elif p_ham_given_msg < p_spam_given_msg:
        return "spam"
    else:
        return "needs human classification"

Let's apply our function to the test set and record the labels on a new column.

In [19]:
test_set["predicted"] = test_set["SMS"].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the true values to measure the accuracy of our spam filter.

In [20]:
correct = 0
total = len(test_set)

for row in test_set.iterrows(): # Iterates over DataFrame rows as (index, Series) pairs
    row = row[1]                # We're not interested in the index
    correct += row["Label"] == row["predicted"]
    
accuracy = correct/total

print("Correct:", correct)
print("Incorrect:", total - correct)
print(accuracy)

Correct: 1095
Incorrect: 19
0.9829443447037702


The accuracy is about 98.29%, which is excellent. Our spam filter looked at 1,114 messages it's never seen in training and correctly classified 1,095 of them. 

# Conclusion

In this project, we managed to build a spam filter for SMS messages using multinomial Naive Bayes algorithm. We originally aimed for an accuracy over 80%, and we managed to surpass that. The spam filter had an accuracy of 98.29% on the test set, which is an excellent result.

Next steps include making the filtering process more complex
* by making the algorithm more sensitive to letter case ("FREE" seems to be more spam-like than "free"), and
* by preserving punctuation ("you won!" and "you won!!!" can appear in different contexts).